<a href="https://colab.research.google.com/github/gyuri2020/PlusProject/blob/master/doctor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nltk

In [ ]:
pip install pymysql

In [ ]:
import nltk
nltk.download()

**논문 추천**

In [ ]:
import math
import operator

import numpy
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt


import pymysql

host = 'database-skku.c6dzc5dnqf69.ap-northeast-2.rds.amazonaws.com'
iid ='admin'
pw = 'tjdrbsrhkseo123'
db_name = 'dongwan'
conn = pymysql.connect(host=host, user= iid, password=pw, db=db_name, charset='utf8')

curs = conn.cursor(pymysql.cursors.DictCursor)

# 데이터 사용 version 1
sql = """SELECT * FROM public.scholar where meshterm != 'null'"""
sql1 = """SELECT * FROM public.all_nor"""
sql2 = """SELECT * FROM medii.doctor_data"""

curs.execute(sql)
rows = curs.fetchall()

data = pd.DataFrame(rows)

curs.execute(sql1)
rows = curs.fetchall()

data1 = pd.DataFrame(rows)

curs.execute(sql2)
rows = curs.fetchall()

data2 = pd.DataFrame(rows)

search_paper_list = list()

data = data.fillna(' ')
print(data.shape)
data['text'] = data['meshterm']
doctor =pd.read_csv('doctordata.csv', low_memory=False,  encoding="UTF-8")

'''
doctor = dict()

def preprocess(text):

    text.lower()

    text = text.replace('-', "HYPHEN")
    text = text.replace('*', "")
    text = text.replace(' / ', ",")
    text = text.replace(' /', ",")
    text = text.replace('/ ', ",")
    text = text.replace(', ', ",")
    text = text.replace(' ,', ",")
    text = text.replace(' ', "JUMP")
    text = text.replace('.', "DOT")

    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)

    filtered_sentence = []
    for w in word_tokens:
        if w not in stop_words and w != ',' and w != '&':
            filtered_sentence.append(w)

    filtered = ', '.join(filtered_sentence)

    return filtered.lower().strip()


data['text'] = data['text'].apply(lambda x: preprocess(x))

def doctorframe(name, text):

    if name in doctor:
        doctor[name] = doctor[name] + ', ' + text

    else:
        doctor[name] = text


data.apply(lambda x: doctorframe(x['name_kor'],x['text']),axis =1)

tempdoctorframe = pd.DataFrame({'name': doctor.keys(),
                             'text': doctor.values()})

tempdoctorframe.to_csv('doctordata.csv',index=False, mode='w')
'''

def get_recommendation(input, option):

    input = preprocess(input)
    name = list(doctor['name'])
    target_index = len(name)
    name.append('target')
    #print(len(doctor.keys()))

    text = list(doctor['text'])
    text.append(input)
    
    #print(input, option)

    doctors = pd.DataFrame({'name': name,
                            'text': text})
    #print(len(doctors['name']))

    tfidf_vector = TfidfVectorizer(min_df =3, max_features = 6000)
    tfidf_matrix = tfidf_vector.fit_transform(doctors['text']).toarray()

    cosine_sim = cosine_similarity(tfidf_matrix)
    cosine_sim_df = pd.DataFrame(cosine_sim, index = doctors.name, columns = doctors.name)
    cosine_sim_df.head()

    for i in range(len(doctors)):
        doctors['text'][i] = doctors['text'][i].replace('jump',' ')
        doctors['text'][i] = doctors['text'][i].replace('hyphen', '-')
        doctors['text'][i] = doctors['text'][i].replace('dot', '.')

    print('-------------------------------------')
    print('검색 질병명 : ' + doctors['text'][target_index])

    paper_similarity = cosine_sim_df['target']
    paper_data = pd.DataFrame({'cosine_similarity': paper_similarity, 'index': np.arange(len(doctors))})

    if option == '1':

        start = 1
        ranking = paper_data.sort_values(by='cosine_similarity', ascending=False)
        topn = 6

        paper_ids = ranking['index'][start:topn]

        std = set(doctors['text'][target_index].split(', '))

        count = start
        recommendation_doctors = list()

        for temp in paper_ids:
            bump = dict()
            bump['name'] = doctors['name'][temp]
            word_target = set(doctors['text'][temp].split(', '))
            words = doctors['text'][temp].split(', ')
            add_keyword = std & word_target
            words_count = {}

            for word in words:
                if word in words_count:
                    words_count[word] += 1
                else:
                    words_count[word] = 1

            sorted_words = sorted([(k, v) for k, v in words_count.items()], key=lambda word_count: -word_count[1])
            keyword = [w for w in sorted_words if w[0] in add_keyword]
            if (len(keyword) >= 5):
                keyword = keyword[0:5]

            bump['simul'] = round(ranking.iloc[count]['cosine_similarity'], 3)
            bump['keyword'] = keyword
            recommendation_doctors.append(bump)
            count = count + 1

        return recommendation_doctors

    if option == '2':

        def score(i):

            target_name = doctors['name'][i]
            person_indexs = data1[data1['name_kor'] == target_name].index

            if(len(person_indexs) == 0):
                return 0

            else:
                person_index = person_indexs[0]
                return data1['scoresum'][person_index] + paper_data['cosine_similarity'][i]*3

        paper_data['total'] = paper_data['index'].apply(lambda x: score(x))

        start = 0
        ranking = paper_data.sort_values(by='total', ascending=False)
        topn = 6

        paper_ids = ranking['index'][start:topn]
        std = set(doctors['text'][target_index].split(', '))

        count = start
        recommendation_doctors = list()

        for temp in paper_ids:

            bump = dict()
            bump['name'] = doctors['name'][temp]

            if bump['name'] == 'target':
                count += 1
                continue

            person_indexs = data2[data2['name_kor'] == bump['name']].index
            person_index = person_indexs[0]
            bump['major'] = data2['major'][person_index]

            person_indexs = data1[data1['name_kor'] == bump['name']].index
            person_index = person_indexs[0]
            bump['belong'] = data1['belong'][person_index]
            bump['paper_count'] = data1['pubnum'][person_index]
            


            word_target = set(doctors['text'][temp].split(', '))
            words = doctors['text'][temp].split(', ')
            add_keyword = std & word_target

            words_count = {}
            for word in words:
                if word in words_count:
                    words_count[word] += 1
                else:
                    words_count[word] = 1

            sorted_words = sorted([(k, v) for k, v in words_count.items()], key=lambda word_count: -word_count[1])
            keyword = [w[0] for w in sorted_words if w[0] in add_keyword]
            if(len(keyword) >= 5):
                keyword = keyword[0:5]

            bump['total'] = round(ranking.iloc[count]['total'], 3)
            bump['simul'] = round(ranking.iloc[count]['cosine_similarity'], 3)
            bump['paper_impact'] = data1['scoresum'][person_index]
            bump['keyword'] = keyword

            recommendation_doctors.append(bump)

            count = count +1

        return recommendation_doctors



input_text = ' '


while(input_text != 'exit'):

    print('\n\n질병명들을 입력해주세요')
    input_text = input()
    print('옵션을 선택하세요')
    print('1. 질병 유사도만 반영 2. 다른 가중치까지 다 반영')
    option = input()
    count = 1

    recom_list = get_recommendation(input_text, option)
    print("---------------------------------------")

    for i in recom_list:
        print(str(count) + '순위')
        for key, value in i.items():
          print(key +' : '+ str(value))
        print('---------------------------------------')

        count += 1

    print('---------------------------------------')









# **임상시험 + 논문**

**논문 함수**

In [ ]:
import math
import operator
import numpy
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import pymysql

host = 'database-skku.c6dzc5dnqf69.ap-northeast-2.rds.amazonaws.com'
iid ='admin'
pw = 'tjdrbsrhkseo123'
db_name = 'dongwan'
conn = pymysql.connect(host=host, user= iid, password=pw, db=db_name, charset='utf8')

curs = conn.cursor(pymysql.cursors.DictCursor)

# 데이터 사용 version 1
sql = """SELECT * FROM public.scholar where meshterm != 'null'"""
sql1 = """SELECT * FROM public.all_nor"""
sql2 = """SELECT * FROM medii.doctor_data"""

curs.execute(sql)
rows = curs.fetchall()

data = pd.DataFrame(rows)

curs.execute(sql1)
rows = curs.fetchall()

data1 = pd.DataFrame(rows)

curs.execute(sql2)
rows = curs.fetchall()

data2 = pd.DataFrame(rows)

search_paper_list = list()

data = data.fillna(' ')
print(data.shape)
data['text'] = data['meshterm']
doctor =pd.read_csv('doctordata.csv', low_memory=False,  encoding="UTF-8")


def get_recommendation(input):

    input = preprocess(input)
    name = list(doctor['name'])
    target_index = len(name)
    name.append('target')

    text = list(doctor['text'])
    text.append(input)

    doctors = pd.DataFrame({'name': name,
                            'text': text})

    tfidf_vector = TfidfVectorizer(min_df =3, max_features = 6000)
    tfidf_matrix = tfidf_vector.fit_transform(doctors['text']).toarray()

    cosine_sim = cosine_similarity(tfidf_matrix)
    cosine_sim_df = pd.DataFrame(cosine_sim, index = doctors.name, columns = doctors.name)
    cosine_sim_df.head()

    for i in range(len(doctors)):
        doctors['text'][i] = doctors['text'][i].replace('jump',' ')
        doctors['text'][i] = doctors['text'][i].replace('hyphen', '-')
        doctors['text'][i] = doctors['text'][i].replace('dot', '.')

    print('-------------------------------------')
    print('검색 질병명 : ' + doctors['text'][target_index])

    paper_similarity = cosine_sim_df['target']
    paper_data = pd.DataFrame({'cosine_similarity': paper_similarity, 'index': np.arange(len(doctors))})


    def score(i):

        target_name = doctors['name'][i]
        person_indexs = data1[data1['name_kor'] == target_name].index

        if(len(person_indexs) == 0):
            return 0

        else:
            person_index = person_indexs[0]
            return data1['scoresum'][person_index] + paper_data['cosine_similarity'][i]*3

    paper_data['total'] = paper_data['index'].apply(lambda x: score(x))

    start = 0
    ranking = paper_data.sort_values(by='total', ascending=False)
    topn = 6

    paper_ids = ranking['index'][start:topn]
    std = set(doctors['text'][target_index].split(', '))

    count = start
    recommendation_doctors = list()

    for temp in paper_ids:

        bump = dict()
        bump['name'] = doctors['name'][temp]

        if bump['name'] == 'target':
            count += 1
            continue

        person_indexs = data2[data2['name_kor'] == bump['name']].index
        person_index = person_indexs[0]
        bump['major'] = data2['major'][person_index]

        person_indexs = data1[data1['name_kor'] == bump['name']].index
        person_index = person_indexs[0]
        bump['belong'] = data1['belong'][person_index]
        bump['paper_count'] = data1['pubnum'][person_index]
        


        word_target = set(doctors['text'][temp].split(', '))
        words = doctors['text'][temp].split(', ')
        add_keyword = std & word_target

        words_count = {}
        for word in words:
            if word in words_count:
                words_count[word] += 1
            else:
                words_count[word] = 1

        sorted_words = sorted([(k, v) for k, v in words_count.items()], key=lambda word_count: -word_count[1])
        keyword = [w[0] for w in sorted_words if w[0] in add_keyword]
        if(len(keyword) >= 5):
            keyword = keyword[0:5]

        bump['total'] = round(ranking.iloc[count]['total'], 3)
        bump['simul'] = round(ranking.iloc[count]['cosine_similarity'], 3)
        bump['paper_impact'] = data1['scoresum'][person_index]
        bump['keyword'] = keyword

        recommendation_doctors.append(bump)

        count = count +1

    return recommendation_doctors



input_text = ' '


while(input_text != 'exit'):

    print('\n\n질병명들을 입력해주세요')
    input_text = input()
    #print('옵션을 선택하세요')
    #print('1. 질병 유사도만 반영 2. 다른 가중치까지 다 반영')
    count = 1

    recom_list = get_recommendation(input_text)
    print("---------------------------------------")

    for i in recom_list:
        print(str(count) + '순위')
        for key, value in i.items():
          print(key +' : '+ str(value))
        print('---------------------------------------')

        count += 1

    print('---------------------------------------')

